In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
import numpy as np

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
import sys
from tqdm import tqdm

In [ ]:
os.chdir(r"C:\Users\hanya\Desktop\MSBD5001 Group Project\Individual Project")

In [ ]:
training_data = pd.read_csv('./train.csv').drop('id', axis=1)
test_data = pd.read_csv('./test.csv').drop('id',axis=1)
training_data['n_jobs'].loc[training_data['n_jobs']==-1] = 16
test_data['n_jobs'].loc[test_data['n_jobs']==-1] = 16
training_label = training_data['time']
data_corr = training_data.corr()['time']

In [ ]:
# 数据平衡
training_data2 = pd.DataFrame(columns=training_data.columns)

In [ ]:
for i in range(10):
    training_data2 = pd.concat([training_data2,
        training_data.loc[np.random.choice(training_label[(training_label<i+1) & (training_label>i)].index, size=30), :]])
training_data2 = pd.concat([training_data2,training_data.loc[np.random.choice(training_label[training_label>10].index, size=400), :]])

In [ ]:
data_corr = training_data2.corr()['time']

In [ ]:
training_data2.index = range(len(training_data2))
training_data = training_data2
training_label = training_data2['time']

In [ ]:
col_drop = data_corr[abs(data_corr)<0.1]
all_data = pd.concat([training_data.drop('time',axis=1), test_data])#.drop('id',axis=1)])
all_data.drop(col_drop.index,axis=1, inplace=True)

In [ ]:
# 特征构造
all_data['n_samples_n_jobs'] = all_data['n_samples']/all_data['n_jobs']
all_data['max_iter_n_jobs'] = all_data['max_iter']/all_data['n_jobs']
all_data['max_iter_n_samples'] = all_data['max_iter']*all_data['n_samples']
all_data['n_samples_n_features'] = all_data['n_samples']*all_data['n_features']
all_data['n_samples_n_features_max_iter'] = all_data['n_samples']*all_data['n_features']*all_data['max_iter']

In [ ]:
all_data = pd.concat([all_data['penalty'], all_data.iloc[:,1:].astype('float')], axis=1)
all_data.dtypes

In [ ]:
all_data1 = all_data
all_data = pd.concat([all_data1['penalty'],(all_data.iloc[:, 1:] - all_data.iloc[:, 1:].mean())/ all_data.iloc[:, 1:].std()], axis = 1)
all_data1 = all_data.iloc[:,1:]

In [ ]:
new_data = pd.get_dummies(all_data)
training_len = 650 #len(training_df_data)-test_training
test_training = len(training_data) - training_len
test_len = len(test_data)

X_train = new_data.iloc[:training_len, :] # training data--cross validation
X_train_test =  new_data.iloc[training_len:training_len+test_training, :] # training data--test
y_train = training_label.iloc[:training_len] # training labels--cross validation
y_train_test = training_label.iloc[-test_training:] # training label--test
X_test = new_data.iloc[-test_len:, :] # test data

In [ ]:
all_data.shape

In [ ]:
def model_nn(X_train, y_train, X_train_test, y_train_test):
    # 200 100
    # 读取数据
    x_train = X_train
    y_train = y_train
    x_test = X_train_test
    y_test = y_train_test

    model = Sequential()
    model.add(Dense(200, activation='relu', input_dim=18))
    model.add(Dropout(0.5))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    adm = Adam(lr=0.01, decay=1e-6)
    model.compile(loss='mean_squared_error',
                  optimizer=adm)

    model.fit(x_train, y_train,
              epochs=1000,
              batch_size=300)
    score = model.evaluate(x_test, y_test, batch_size=60)
    return model, score

In [ ]:
u, w = model_nn(X_train, y_train, X_train_test, y_train_test)

In [ ]:
pd.DataFrame(u.predict(X_test)).to_csv('test_nn_balanced1.csv')

In [ ]:
w

In [ ]:
pd.concat([pd.DataFrame(u.predict(X_train_test), index=y_train_test.index), y_train_test], axis=1)